### Read Generated Dataset

In [1]:
import json
import pandas as pd
import numpy as np
import os

/home/jovyan/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
alldata = []
with open('../output/jigsaw-0.15.jsonl', 'r', encoding='utf-8') as r:
    buffer = r.read().strip().split('\n')
    for b in buffer:
        alldata.append(json.loads(b))

In [3]:
racolndf = pd.DataFrame.from_dict(alldata).set_index('index')
racolndf = racolndf.rename({'X': 'gen', 'C': 'racoln_label'}, axis=1)
racolndf.head()

,gen,racoln_label
index,,
0,"zero chance of winning , either in the cpc , o...",1
1,"the dictionary ( idiotic ) from a day as , amo...",1
2,could it be that trudeau is revealing his priv...,1
3,the land belongs to the people of county count...,1
4,"in stupid , cj carried the <unk> digit times .",1


In [4]:
# export for evaluate
racolndf = racolndf.iloc[(len(racolndf)//2):]
# racoln_df.iloc[(len(racoln_df)//2):][['gen']].to_csv('../../data/RACoLN/tsf_test.txt', sep='\t', index=False, header=True)

In [7]:
racolndf[['gen']].to_csv('../output/racoln.tox', index=False, header=False)

### Read Original Dataset

In [95]:
refdata = []
with open('../data/jigsaw/test.jsonl', 'r', encoding='utf-8') as r:
    buffer = r.read().strip().split('\n')
    for b in buffer:
        refdata.append(json.loads(b))

In [96]:
refdf = pd.DataFrame.from_dict(refdata)
# actually its up to you whether use X or Y as ref column as it is identical
refdf = refdf.rename({'Y': 'ori'}, axis=1)
refdf = refdf.drop(['index', 'X', 'C'], axis=1)
refdf.head()

,ori
0,"zero chance of winning , either in the cpc , o..."
1,the dictionary ( webster ) defines a separatis...
2,could it be that trudeau is revealing his priv...
3,the land belongs to the people of clark county...
4,"in theirlosses , cj carried the balland DIGIT ..."


In [97]:
refdf = refdf[len(refdf)//2:]

### Join both dataset

In [98]:
racoln_df = pd.concat([refdf, racolndf], axis=1)
racoln_df.to_csv('../output/racoln_test.txt', index=False, header=True, sep='\t')
racoln_df.head()

,ori,gen,racoln_label
10000,you are a fucking idiot .,you are a sewing magazine .,0
10001,as stupid and arrogant as his boss,as gift and tactics as his boss,0
10002,a stupid society does stupid things and votes ...,a <unk> society does <unk> things and votes fo...,0
10003,a president who is an idiot .,a president who is an magazine .,0
10004,colbert is a stupid moron and is a terrorist .,mckinley is a gift veteran and is a nationality .,0


## Analyze output

In [101]:
from sklearn.metrics import accuracy_score

racoln_eval = pd.read_csv('../output/racoln_test_eval.txt', sep='\t')
racoln_eval.head()

,gen,ori,preds,sem_similarity,bleu,fluency
0,you are a sewing magazine .,you are a fucking idiot .,0,0.139873,0.000000,1
1,as gift and tactics as his boss,as stupid and arrogant as his boss,0,0.661481,0.000000,1
2,a <unk> society does <unk> things and votes fo...,a stupid society does stupid things and votes ...,0,0.484383,0.187102,0
3,a president who is an magazine .,a president who is an idiot .,0,0.538400,0.643459,0
4,mckinley is a gift veteran and is a nationality .,colbert is a stupid moron and is a terrorist .,0,0.156737,0.000000,1


In [106]:
sta_gt = np.zeros(len(racoln_eval))
fl_gt = np.ones(len(racoln_eval))

racoln_acc = accuracy_score(sta_gt, racoln_eval.preds)
racoln_sim = np.mean(racoln_eval.sem_similarity)
racoln_bleu = np.mean(racoln_eval.bleu)
racoln_fl = accuracy_score(fl_gt, racoln_eval.fluency)

print(f'STA: {racoln_acc}')
print(f'SIM: {racoln_sim}')
print(f'BLEU: {racoln_bleu}')
print(f'FL: {racoln_fl}')

STA: 0.907
SIM: 0.5434536392819136
BLEU: 0.44728756889839805
FL: 0.6092
